## Imports

In [ ]:
from pathlib import Path
import warnings

import h5py
import numpy as np
import scipy as sp
import skimage as ski
import matplotlib.pyplot as plt
import torch
from ipywidgets import interact

from fusion import optimize, optimize_with_continuous_time
from baseline import baseline
from utils import (
    RESOURCES_PATH,
    hyperspectral2RGB,
    hyperspectral2RGBvolume,
    bin_data,
    get_discrete_time_decay,
)

# RESOURCES_PATH = Path("/Users/federicosimoni/Library/Mobile Documents/com~apple~CloudDocs/Università/Tesi/Code/CS-FLIM_lab/20240703/kidney")

CMOS_PATH = RESOURCES_PATH / "20240703" / "kidney2_520_4m_rec_FOVcorrected.mat"
SPC_PATH = RESOURCES_PATH / "20240703" / "520_kidney2_550_550_610_SPC_raw_proc_tlxy.mat"
LAMBDA_PATH = RESOURCES_PATH / "20240703" / "610_Lambda_L16.mat"

# CMOS_PATH = RESOURCES_PATH / "beads" / "3beads_triangle_w4_rec_Hil2D_FOVcorrected.mat"
# SPC_PATH = RESOURCES_PATH / "beads" / "480_3beads_triangle_505_500_575_SPC_raw_proc_tlxy.mat"
# LAMBDA_PATH = RESOURCES_PATH / "beads" / "575_Lambda_L16.mat"

# CMOS_PATH = RESOURCES_PATH / "kidney_cells_520_610_w4_rec_Hil2D_FOVcorrected.mat"
# SPC_PATH = RESOURCES_PATH / "520_kidneyCells_550_550_610_SPC_raw_proc_tlxy.mat"
# LAMBDA_PATH = RESOURCES_PATH / "610_Lambda_L16.mat"

XY_DIM = 256
ZOOM_FACTOR = 0
Z_START = 6
Z_END = 12

## Data loading

In [ ]:
with h5py.File(CMOS_PATH, "r") as f:
    cmos = np.array(f["I"])

if cmos.ndim == 3:
    cmos = np.transpose(cmos, (1, 2, 0))
    cmos = ski.transform.resize(cmos, (XY_DIM, XY_DIM, cmos.shape[2]))
    cmos = np.transpose(cmos, (2, 1, 0))
else:
    cmos = ski.transform.resize(cmos, (XY_DIM, XY_DIM))
    cmos = np.transpose(cmos, (1, 0))

cmos = cmos[Z_START:Z_END]
cmos = cmos / cmos.max()

original_spc = sp.io.loadmat(SPC_PATH)["im"]
original_spc[:, :, 0, 0] = original_spc[:, :, 1, 0]

original_t = np.squeeze(sp.io.loadmat(SPC_PATH)["t"])
lam = np.squeeze(sp.io.loadmat(LAMBDA_PATH)["lambda"])

if ZOOM_FACTOR > 0:
    new_x = slice(XY_DIM // 2 - XY_DIM // ZOOM_FACTOR, XY_DIM // 2 + XY_DIM // ZOOM_FACTOR)
    new_y = slice(XY_DIM // 2 - XY_DIM // ZOOM_FACTOR, XY_DIM // 2 + XY_DIM // ZOOM_FACTOR)
    cmos = cmos[:, new_x, new_y]

## Time binning and initial plots

In [ ]:
# cut the time (first)
curve = np.squeeze(np.sum(original_spc, axis=(1, 2, 3)))
val = np.max(curve)
pos_max = np.argmax(curve)
curve = curve[pos_max:]
# b = np.argmin(abs(curve - curve[0] * (1 - np.exp(-1 / 1))))
b = 1024

spc = original_spc[pos_max: (pos_max + b)]
t = original_t[pos_max: (pos_max + b)]

# data binning (after cut)
t, spc, dt = bin_data(spc, t, 0.4)
spc = spc / np.max(spc)

time_decay = np.mean(spc, axis=(1, 2, 3))

_, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(t, time_decay)
ax[0].set_title(f"Global time decay - {len(t)} temporal points")
ax[0].set_xlabel("Time [ns]")
ax[0].grid()

ax[1].plot(lam, spc.mean(axis=(0, 2, 3)))
ax[1].set_title(f"Global spectrum - {len(lam)} channels")
ax[1].set_xlabel("Wavelength [nm]")
ax[1].grid()

plt.tight_layout()
plt.show()

In [ ]:
initial_spectrums = np.mean(spc, axis=0)
min_spectrums, max_spectrums = np.min(initial_spectrums), np.max(initial_spectrums)

initial_times = np.mean(spc, axis=1)
min_times, max_times = np.min(initial_times), np.max(initial_times)


def spc_spectrum_time_in_a_point(z=0, i=0, j=0):
    _, ax = plt.subplots(1, 4, figsize=(12, 3))
    ax[0].imshow(cmos[z], cmap="gray", vmin=0, vmax=1)
    ax[0].set_title(f"CMOS in {z}")

    ax[1].imshow(hyperspectral2RGB(lam, spc.mean(axis=0)))
    ax[1].scatter([j], [i], c="w")
    ax[1].set_title(f"Colored with spectrum")

    ax[2].plot(lam, initial_spectrums[:, i, j])
    ax[2].set_ylim(min_spectrums, max_spectrums)
    ax[2].set_title(f"Spectrum in ({i},{j})")
    ax[2].grid()

    ax[3].plot(t, initial_times[:, i, j])
    ax[3].set_ylim(min_times, max_times)
    ax[3].set_title(f"Time in ({i},{j})")
    ax[3].grid()

    plt.tight_layout()
    plt.show()


interact(
    spc_spectrum_time_in_a_point,
    z=(0, cmos.shape[0] - 1, 1),
    i=(0, spc.shape[-2] - 1, 1),
    j=(0, spc.shape[-1] - 1, 1),
);

## Fusion through optimization

In [ ]:
# spectral_time uses the matrix cosine loss if > 0
# spectral and time use cosine loss if > 0

weights = {
    "spectral": 0.1,
    "time": 0.1,
    "spatial": 0.8,
    "spectral_time": 0.0,
}

x = optimize(
    spc,
    cmos,
    weights=weights,
    lr=0.01,
    iterations=1000,
    device="cpu",
    init_type="random",
    mask_initializations=False,
    mask_gradients=False,
    non_neg=False,
    return_numpy=True,
)

## Fusion through optimization with continuous time

In [ ]:
weights = {
    "spectral": 0.1,
    "time": 2,
    "spatial": 1,
    "spectral_time": 0.0,
}

x = optimize_with_continuous_time(
    spc,
    cmos,
    weights=weights,
    t=np.arange(0.0001, dt * len(t), dt),
    n_decays=1,
    lr=0.01,
    iterations=100,
    device="cpu",
    init_type="random",
    mask_initializations=False,
    mask_gradients=False,
    non_neg=False,
    return_numpy=False,
)

# If you use optimize with continuous time, you can use this cell to get the discrete time decay
from fusion import _get_masks

_, cmos_mask = _get_masks(torch.from_numpy(spc), torch.from_numpy(cmos))
x = get_discrete_time_decay(x.cpu().detach(), np.arange(0.0001, dt * len(t), dt), cmos_mask)
x = x.cpu().detach().numpy()

## Fusion through baseline

In [ ]:
x_baseline = baseline(cmos, spc, device="cpu", return_numpy=True)

## Visualize results

In [ ]:
slices_rgb = hyperspectral2RGBvolume(lam, np.mean(x, axis=0))
slices_rgb_baseline = hyperspectral2RGBvolume(lam, np.mean(x_baseline, axis=0))

means_spectrums = np.mean(x, axis=(0, 3, 4)).T
min_mean_spectrum = np.min(means_spectrums)
max_mean_spectrum = np.max(means_spectrums)

means_times = np.mean(x, axis=(1, 3, 4)).T
min_mean_times = np.min(means_times)
max_mean_times = np.max(means_times)

zxy = np.sum(x, axis=(0, 1))
zxy /= zxy.max()

def plot_across_z(z=0, i=0, j=0):
    _, ax = plt.subplots(2, 3, figsize=(12, 8))
    ax[0, 0].imshow(slices_rgb_baseline[z])
    ax[0, 0].scatter([j], [i], c="w")
    ax[0, 0].set_title(f"Baseline z={z}")

    ax[0, 1].plot(lam, means_spectrums[z])
    ax[0, 1].set_ylim(min_mean_spectrum, max_mean_spectrum)
    ax[0, 1].set_title(f"Global Spectrum in z={z}")
    ax[0, 1].grid()

    ax[0, 2].plot(t, means_times[z])
    ax[0, 2].set_ylim(min_mean_times, max_mean_times)
    ax[0, 2].set_title(f"Global Time in z={z}")
    ax[0, 2].grid()

    reconstructed_spectrums = np.mean(x[:, :, z, :, :], axis=0)
    baseline_spectrums = np.mean(x_baseline[:, :, z, :, :], axis=0)
    min_spectrums = np.min(reconstructed_spectrums)
    max_spectrums = np.max(reconstructed_spectrums)

    reconstructed_times = np.mean(x[:, :, z, :, :], axis=1)
    baseline_times = np.mean(x_baseline[:, :, z, :, :], axis=1)
    txy_spc = np.mean(spc, axis=1)
    lxy_spc = np.mean(spc, axis=0)
    min_times = np.min(reconstructed_times)
    max_times = np.max(reconstructed_times)

    ax[1, 0].imshow(slices_rgb[z])
    ax[1, 0].scatter([j], [i], c="w")
    ax[1, 0].set_title(f"Reconstruction Spectral Colored z={z}")

    normalized_spectrum_recon = reconstructed_spectrums[:, i, j] / (0.00001 + reconstructed_spectrums.max())
    normalized_spectrum_baseline = baseline_spectrums[:, i, j] / (0.00001 + baseline_spectrums.max())
    normalized_spectrum_spc = lxy_spc[:, i // 8, j // 8] / (0.00001 + initial_spectrums.max())
    ax[1, 1].plot(lam, normalized_spectrum_recon, label="Datafusion")
    ax[1, 1].plot(lam, normalized_spectrum_baseline, label="Baseline") 
    ax[1, 1].plot(lam, normalized_spectrum_spc, label="SPC")
    ax[1, 1].set_title(f"Spectrum in ({i},{j})")
    # ax[1, 1].set_ylim(min_spectrums, max_spectrums)
    ax[1, 1].legend(loc="upper right")
    ax[1, 1].grid()

    normalized_time_recon = reconstructed_times[:, i, j] / (0.00001 + reconstructed_times.max())
    normalized_time_baseline = baseline_times[:, i, j] / (0.00001 + baseline_times.max())
    normalized_time_spc = txy_spc[:, i // 8, j // 8] / (0.00001 + txy_spc.max())
    ax[1, 2].plot(t, normalized_time_recon, label="Datafusion")
    ax[1, 2].plot(t, normalized_time_baseline, label="Baseline")
    ax[1, 2].plot(t, normalized_time_spc, label="SPC")
    ax[1, 2].set_title(f"Time in ({i},{j})")
    # ax[1, 2].set_ylim(min_times, max_times)
    ax[1, 2].legend(loc="upper right")
    ax[1, 2].grid()

    plt.tight_layout()
    plt.show()


interact(
    plot_across_z,
    z=(0, len(zxy) - 1, 1),
    i=(0, x.shape[-2] - 1, 1),
    j=(0, x.shape[-1] - 1, 1),
);

## Save data

In [ ]:
SAVE_PATH = RESOURCES_PATH / "520_kidney2_550_550_610_SPC_raw_proc_tlxy_fused_PROVA.npz"
if not SAVE_PATH.is_file():
    np.savez(SAVE_PATH, x=x.astype(np.float16), t=t, lam=lam, weights=weights)
else:
    warnings.warn("File name already exists!")